In [1]:
import os

%pwd
%cd '/home/cs/Chicken-Disease-Classification'


/home/cs/Chicken-Disease-Classification


In [2]:
from dataclasses import  dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model: Path
    params_image_size: list
    training_data:Path
    params_epochs: int
    params_batch_size: int
    params_is_agumentation: bool



@dataclass(frozen=True)
class ModelCallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path


In [3]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directory


In [7]:
class ConfigurationManager:
    def __init__(self,config_file_path=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directory([self.config.artifacts_root])

    
   
    def get_prepare_callback_config(self):
        config=self.config.prepare_callbacks
        model_checkpoint_dir=os.path.dirname(config.checkpoint_model_filepath)
        create_directory([Path(model_checkpoint_dir),Path(config.tensorboard_root_log_dir)])

        prepare_callback_config=ModelCallbackConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
            
        )

        return prepare_callback_config
    def get_training_config(self):
        training=self.config.training
        prepare_base_model=self.config.prepare_base_model
        params=self.params
        training_data=os.path.join(self.config.data_ingestion.unzip_dir,'Chicken-fecal-images')
        create_directory([Path(training.root_dir)])


        training_config=TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model=Path(prepare_base_model.updated_base_model_path),
            params_image_size=params.IMAGE_SIZE,
            training_data=training_data,
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_agumentation=params.AUGMENTATION
                    
        )
        return training_config


        

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time


2024-12-14 15:38:03.407709: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
class Training:
    def __init__(self,config: TrainingConfig):
        self.config=config
    def get_base_model(self):
        self.model=tf.keras.models.load_model(
            self.config.updated_base_model
        )
    def train_valid_generator(self):
        datagenerator_kwargs=dict(rescale=1/255,validation_split=0.20)
        dataflow_kwargs=dict(target_size=self.config.params_image_size[:-1],
                             batch_size=self.config.params_batch_size,
                             interpolation='bilinear')
        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
        **datagenerator_kwargs
        )
        self.valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_agumentation:
            train_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator=valid_datagenerator

        self.train_generator=train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='training',
            shuffle=True,
            **dataflow_kwargs
        )
    @staticmethod
    def save_model(path: Path,model: tf.keras.Model):
        model.save(path)
    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [10]:
try:
    config=ConfigurationManager()
    training_config=config.get_training_config()
    training=Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e

[2024-12-14 15:38:29,442: INFO: common:Reading yaml file from:config/config.yaml]
[2024-12-14 15:38:29,684: INFO: common:Reading yaml file from:params.yaml]
[2024-12-14 15:38:29,722: INFO: common:directory created at:artifacts]
[2024-12-14 15:38:29,723: INFO: common:directory created at:artifacts/training]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.


2024-12-14 15:38:35.743731: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-12-14 15:38:38.050698: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2024-12-14 15:38:40.352173: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2024-12-14 15:38:41.988349: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51380224 exceeds 10% of free system memory.
2024-12-14 15:38:42.150431: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2024-12-14 15:38:42.678853: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10

19/19 [==============================] - ETA: 0s - loss: 11.4178 - accuracy: 0.5541

2024-12-14 15:41:00.245320: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


19/19 [==============================] - 186s 10s/step - loss: 11.4178 - accuracy: 0.5541 - val_loss: 6.7010 - val_accuracy: 0.6094
